In [126]:
import arcgis
from arcgis.apps import storymap
from arcgis.features import FeatureLayerCollection

In [129]:
import arcgis
from arcgis.gis import GIS
from arcgis.apps import storymap
from getpass import getpass
from arcgis.features import FeatureLayer

# Create a connection to your portal for publishing
gis = GIS("https://www.arcgis.com", "ezhatcher", getpass())

········


In [130]:
seedappItem = gis.content.get("2a51cb9cc9d54b57a52c67d8efdb5eba") #Hoonah Alaska Template App

In [131]:
contentManager = arcgis.gis.ContentManager(gis)
clones = contentManager.clone_items(items=[seedappItem],search_existing_items=False)
clones
clonedStorymap = storymap.JournalStoryMap(clones[0])

In [132]:
clonedStorymap.save(title="Nome Storymap Test", description="Test cloning script result", tags="Test,Storymap")
sections = clonedStorymap.properties["values"]["story"]["sections"]
community = 'Nome'

In [147]:
#Access the URL for the main photo from the Photo Link feature class
photo_base = gis.content.get("6d9d437bbe2d48bfa87b4d9fca41f80d")
photos = photo_base.layers
photo_lyr = photos[0]
find_url = photo_lyr.query(where="CommunityName = '" + str(community) + "'",out_fields='PhotoLink')
url_attributes = find_url.df
main_pic = url_attributes.PhotoLink[0]

In [134]:
#Modify the main title and photo in the main Table of Contents panel
panel_toc = sections[0]
panel_toc["title"] = '<span style="font-size:40px">' + str(community) + ', Alaska</span>'
panel_toc["media"]["image"]["url"] = main_pic
clonedStorymap.save()

True

In [135]:
def access_lyr(content,out_fld):
    base = content.layers
    lyr = base[0]
    topic_data = lyr.query(where="CommunityName = '" + str(community) + "'",out_fields=out_fld)
    return topic_data

In [136]:
#Access the culture and history from the culture/history feature class (IF THE PANEL ORDER CHANGES, MAKE SURE INDEX STILL APPLIES)
panel_ch = sections[1]
ch_ft = gis.content.get("3f99cf21a0da42c4bf8b15632e958df9")
culture_lookup = access_lyr(ch_ft,'CommunityCulture')
culture = culture_lookup.df.CommunityCulture[0]
history_lookup = access_lyr(ch_ft,'CommunityHistory')
history = history_lookup.df.CommunityHistory[0]

In [137]:
#Populate the panel with the new culture history info
panel_ch["content"] = '<style type="text/css">.dcra-blue {\n     display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 0px 4px;\n     border-radius: 0px;\n}\n</style>\n<p><a class="dcra-blue" data-storymaps="MJ-ACTION-1524783785796" data-storymaps-type="navigate">Back to Table of Contents</a></p>\n\n<p>&nbsp;<p>'+ history + '&nbsp;</p>\n\n<p>\n\n<p>&nbsp;<p>'+ culture + '&nbsp;</p>\n'
clonedStorymap.save()

True

In [138]:
#Access the demographics panel and clone the webmap/update map filters
panel_demog = sections[2]
current_webmap = gis.content.get(panel_demog["contentActions"][1]["media"]["webmap"]["id"])
wm = arcgis.mapping.WebMap(webmapitem=current_webmap)
wm_properties = {'title':'OpenData_Storymap_' + str(community),'snippet':'Test webmap clone','tags':['dcra','test']}
wm.save(wm_properties,folder='Storymap Webmaps') 

<Item title:"OpenData_Storymap_Nome" type:Web Map owner:ezhatcher>

In [139]:
#Change the webmap in the demographics panel
new_map_search = gis.content.search('OpenData_Storymap_' + str(community), item_type="Web Map")
webmap = new_map_search[0]
new_webmap = webmap.id
panel_demog["contentActions"][1]["media"]["webmap"]["id"] = new_webmap

In [140]:
upd_webmap = gis.content.get(new_webmap)
panel_wm = arcgis.mapping.WebMap(webmapitem=upd_webmap)

In [141]:
#Set up filters in the web map to fit the community (IF LAYERS ARE ADDED OR REMOVED, UPDATE THIS CELL)
def_expr = "CommunityName = '" + str(community) + "'"
def_expr2 = "City = '" + str(community) + "'"
conv2_lyrs = [9, 10, 12]
for i in range(7):
    wm_lyrs = panel_wm.layers[i]
    wm_lyrs["layerDefinition"]["definitionExpression"] = def_expr
panel_wm.layers[8]['layerDefinition']['definitionExpression'] = def_expr2
for n in conv2_lyrs:
    wm_lyrs = panel_wm.layers[n]
    wm_lyrs["layers"][0]["layerDefinition"]["definitionExpression"] = def_expr
panel_wm.update()

True

In [142]:
#Adjust Culture History layer y-offset
panel_wm.layers[6]["layerDefinition"]["drawingInfo"]["renderer"]["symbol"]["yoffset"] = 16
panel_wm.update()

True

In [143]:
#Set up zoom scale (x max and min, y max and min) for map actions
bound_expr = "NAME = '" + str(community) + "'"
bounds_ft = gis.content.get("8499c3bc4ff141588f5c9ab1db99b0fb").layers[0]
x_min = bounds_ft.query(where=bound_expr,out_fields='EXT_MIN_X').df.EXT_MIN_X[0]
x_max = bounds_ft.query(where=bound_expr,out_fields='EXT_MAX_X').df.EXT_MAX_X[0]
y_min = bounds_ft.query(where=bound_expr,out_fields='EXT_MIN_Y').df.EXT_MIN_Y[0]
y_max = bounds_ft.query(where=bound_expr,out_fields='EXT_MAX_Y').df.EXT_MAX_Y[0]

geometry = url_attributes.SHAPE[0]
anchor_x = geometry["x"]
anchor_y = geometry["y"]

#Function to define the primary zoom scale of each panel based on the minimum bounding envelope feature service
# item id = "8499c3bc4ff141588f5c9ab1db99b0fb" using previously defined zoom scale
def main_extent(p):
    ext1 = p['contentActions'][1]["media"]["webmap"]["extent"]
    ext1["xmin"] = x_min
    ext1["xmax"] = x_max
    ext1["ymin"] = y_min
    ext1["ymax"] = y_max
    anch_pt = p["contentActions"][1]["media"]["webmap"]["popup"]["anchorPoint"]
    anch_pt["x"] = anchor_x
    anch_pt["y"] = anchor_y
    clonedStorymap.save()

In [144]:
#Demographics Panel Updates
#Update the zoom extent using established zoom scale from 255
main_extent(panel_demog)

demog_ext2 = panel_demog["media"]["webmap"]["extent"]
demog_ext2["xmin"] = x_min
demog_ext2["xmax"] = x_max
demog_ext2["ymin"] = y_min
demog_ext2["ymax"] = y_max

panel_demog["media"]["webmap"]["popup"]["anchorPoint"]["x"] = anchor_x
panel_demog["media"]["webmap"]["popup"]["anchorPoint"]["y"] = anchor_y

clonedStorymap.save()

True

In [145]:
url_attributes.SHAPE[0]["x"]

-545131.2928999998

In [146]:
#Education Panel Updates
panel_edu = sections[4]
main_extent(panel_edu)
school_ft = gis.content.get("1f8928a4bae646ef884be3b6a0b8bae8")
panel_edu

{'title': '<span style="font-size:26px">Education</span>',
 'content': '<p>&nbsp;</p>\n\n<p>\n<style type="text/css">.blue {\n     display: inline-block;\n     background-color: #3182bd;\n     border-color: #3182bd !important;\n     color: #fff !important;\n     padding: 1px 3px;\n     border-radius: 12px;\n</style>\n<style type="text/css">.dcra-blue {\n     display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 3px 8px;\n     border-radius: 5px;\n</style>\n</p>\n<style type="text/css">.btn-orange {\n    display: inline-block;\n    background-color: #f0ad4e;\n    border-color: #f0ad4e !important;\n    color: #fff !important;\n    padding: 0px 2px;\n    border-radius: 12px;\n}\n</style>\n<p><a class="dcra-blue" data-storymaps="MJ-ACTION-1524785467480" data-storymaps-type="navigate">Back to Table of Contents</a><br>\n<style type="text/css">\n</style>\n</p>\n\n<p>&nbsp;</p>\n\n<p>&nbsp;</p>\n\n<p>&nbsp;<